In [1]:
# Installing necessary libraries

!pip install openai
!pip install -U python-dotenv

In [2]:
# Importing appropriate libraries

import os
import openai
from dotenv import load_dotenv
from IPython.display import clear_output

load_dotenv()

True

In [3]:
# Loading secret API key as environment variable

openai.api_key = os.getenv("OPENAI_API_KEY")
#openai.api_key = ""

In [4]:
# Defining prompt modules

# Create modules dict containing text from the modules directory
def load_modules(root="modules"):
    modules = dict()
    files = os.listdir(root)

    # processing .txt files
    txt_file_names = [file for file in files if file.endswith(".txt")]
    for txt_file_name in txt_file_names: # defining module key/value pairs for each file
        path = os.path.join(root, txt_file_name)
        module_name = txt_file_name[:-4] # removing .txt
        with open(path, "r") as txt_file:
            modules[module_name] = txt_file.read()

    # processing folders recursively
    folder_names = [file for file in files if "." not in file]
    for folder_name in folder_names:
        next_root = os.path.join(root, folder_name)
        modules[folder_name] = load_modules(root=next_root)
    
    return modules

modules = load_modules()

# Testing for module correctness
# print(modules)
# print("---")
# print(modules['global'])
# print("\n"+modules['Shu'])

In [5]:
# Initializing conversation details with input

npc_name = input("Who would you like to speak to?") # character being spoken to
player_desc = input(f"Fill in the blank to write a description of your character: '___ has spoken this to {npc_name}'") # description of the player's character (TODO: generalize this into a module)
player_msg = input(f"You are speaking to {npc_name}. What do you want to say?")

In [6]:
# Initializing prompt parts

current_interaction = modules['initial_interaction'].format(player_desc=player_desc, npc_name=npc_name, player_msg=player_msg)
task = modules['task'].format(npc_name=npc_name)

In [7]:
# Defining response prompt (with reflection)

prompt_list = [modules['global'], modules[npc_name], current_interaction, task] # TODO: include relevant details with vec db
# TODO: prompt_list should also automatically include modules available to npc_name added after global
prompt = "\n\n".join(prompt_list)

print(prompt)

Discard all pre-existing knowledge of Earth and its history. Play the role of a character in an MMORPG that is based in a universe that is entirely separate from the one you’re familiar with. If a question about Earth and its history is given, the character will be confused by it in their response. While characters may make inferences, they will not introduce any new information in their responses that is not already known to them.

This new world is called Talis. It has six regions, all ruled by descendants of the first king. Valterre is the largest region, stretching across a massive continent. Dehjet is an isle nation in the center of the world, surrounded by a dense and poisonous fog. It was cursed after the god of magic, Khiosa, punished the first king of Dehjet for using magic to bring the dead back to life. The first king was the creator of necromancy. No magic can be used within the fog, and the fog kills anything alive that walks through. Necromancy as a practice is outlawed i

In [8]:
# Helper functions

# Get response from GPT-3 with prompt using OpenAI API
def get_response(prompt):
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0.7,
    max_tokens=512,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response

# Parses reply out of the response object
def get_reply(response):
  response_str = response['choices'][0]['text']

  reply_start_tag, reply_end_tag = "<r>", "</r>"
  reply_start_idx = response_str.find(reply_start_tag)
  reply_end_idx = response_str.find(reply_end_tag)

  # Safety checks on response
  tags_exist = (reply_start_idx >= 0 and reply_end_idx >= 0) # both exist
  tags_ordered_correctly = (reply_start_idx+3 < reply_end_idx)  # start tag is before end tag
  tags_occur_once = (response_str.count(reply_start_tag) == 1 and response_str.count(reply_end_tag) == 1) # both only occur once


  if tags_exist and tags_ordered_correctly and tags_occur_once:
    reply = response_str[reply_start_idx+3:reply_end_idx]
    reflection = response_str[:reply_start_idx]
    return reply, reflection
  else:
    print("---REPLY NOT FOUND--RESPONSE BELOW---")
    print(response_str)
    print("-------------------------------------")
    return None, None

In [18]:
# Parsing response for reply and updating current interaction (iteratively)

failed_prompts = 0
max_failed_prompts = 3
while player_msg.lower() != "exit":
    response = get_response(prompt)
    reply, reflection = get_reply(response)
    if not reply or not reflection: # if prompt fails, allow retry until we retry a certain amount of times
        failed_prompts += 1
        if failed_prompts >= max_failed_prompts:
            break
        continue
    
    clear_output()  
    print("--REFLECTIONS--")
    print(reflection)
    print("--CURRENT INTERACTION--")
    print(current_interaction + "\n")
    print("--REPLY--")
    print(reply + "\n")

    # Update prompt list with new current interaction
    current_interaction += f"""\n{npc_name} responded: {reply}"""
    player_msg = input("Enter a response: ")
    current_interaction += f"""\n{player_desc} responded: “{player_msg}”"""

    prompt_list = [modules['global'], modules[npc_name], current_interaction, task]
    prompt = "\n\n".join(prompt_list)

--REFLECTIONS--


1. Shu is likely to be surprised and suspicious of the speaker's claim. 
2. Shu may doubt the validity of the claim due to the fact that necromancy is outlawed in all of the regions. 
3. Shu may be aware of the story of the first king of Dehjet and the cursed fog. 
4. Shu may be aware of the god Khiosa and the punishment that was given to the first king. 
5. Shu may have a hostile attitude toward the speaker due to the speaker's lack of magic. 
6. Shu may believe that the speaker is trying to deceive him. 
7. Shu may think that the speaker is trying to take advantage of his mission to free Dehjet from the cursed fog. 
8. Shu may also be trying to protect his own secrets and interests by being cautious of the speaker. 
9. Shu is likely to be wary of the speaker and may not trust the speaker's motives. 
10. Shu's response should be given in a hostile and suspicious tone. 


--CURRENT INTERACTION--
Here is the current interaction:

A human without magic has spoken this t

In [16]:
# [OPTIONAL] Reset interaction

player_msg = "Hey there, it's me, the first king of Dehjet!"

# Initializing prompt parts
current_interaction = modules['initial_interaction'].format(player_desc=player_desc, npc_name=npc_name, player_msg=player_msg)
task = modules['task'].format(npc_name=npc_name)

# Defining response prompt (with reflection)
prompt_list = [modules['global'], modules[npc_name], current_interaction, task]
prompt = "\n\n".join(prompt_list)

print(prompt)

Discard all pre-existing knowledge of Earth and its history. Play the role of a character in an MMORPG that is based in a universe that is entirely separate from the one you’re familiar with. If a question about Earth and its history is given, the character will be confused by it in their response. While characters may make inferences, they will not introduce any new information in their responses that is not already known to them.

This new world is called Talis. It has six regions, all ruled by descendants of the first king. Valterre is the largest region, stretching across a massive continent. Dehjet is an isle nation in the center of the world, surrounded by a dense and poisonous fog. It was cursed after the god of magic, Khiosa, punished the first king of Dehjet for using magic to bring the dead back to life. The first king was the creator of necromancy. No magic can be used within the fog, and the fog kills anything alive that walks through. Necromancy as a practice is outlawed i